In [ ]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import random
import os
import pandas as pd
from torchvision.io import read_image
from torchvision import transforms
from torchvision import models

In [ ]:
lr = 1e-3
batch_size = 64

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

trans = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.491, 0.482, 0.447), (0.247, 0.243, 0.262))
])

root = '' ## 이미지 경로

original_trainset = datasets.ImageFolder(root=root+"/origin-cifar-10/train", transform=trans)
original_testset = datasets.ImageFolder(root=root+"origin-cifar-10/test", transform=trans)
svd_trainset = datasets.ImageFolder(root=root+"/svd-cifar-10/train", transform=trans)
svd_testset = datasets.ImageFolder(root=root+"/svd-cifar-10/test", transform=trans)
svd4_trainset = datasets.ImageFolder(root=root+"/svd4-cifar-10/train", transform=trans)
svd4_testset = datasets.ImageFolder(root=root+"/svd4-cifar-10/test", transform=trans)

o_trainloader = DataLoader(original_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
o_testloader = DataLoader(original_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s_trainloader = DataLoader(svd_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s_testloader = DataLoader(svd_testset, batch_size=batch_size, shuffle=False, drop_last=False)
s4_trainloader = DataLoader(svd4_trainset, batch_size=batch_size, shuffle=True, drop_last=True)
s4_testloader = DataLoader(svd4_testset, batch_size=batch_size, shuffle=False, drop_last=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model1 = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model2 = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model3 = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)

model1.fc = nn.Linear(in_features=model1.fc.in_features, out_features=10, bias=True)
model2.fc = nn.Linear(in_features=model2.fc.in_features, out_features=10, bias=True)
model3.fc = nn.Linear(in_features=model3.fc.in_features, out_features=10, bias=True)

nn.init.xavier_normal_(model1.fc.weight)
nn.init.xavier_normal_(model2.fc.weight)
nn.init.xavier_normal_(model3.fc.weight)

model1.to(device)
model2.to(device)
model3.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

In [ ]:
def train_loop(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    loss_sum = 0
    for batch, (X, y) in enumerate(dataloader):
        pred, _ = model(X.to(device))
        loss = criterion(pred, y.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            loss_sum += loss
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

    return loss_sum / size

def test_loop(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    hit, loss = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X.to(device))
            loss += criterion(pred, y.to(device)).item()
            hit += (pred.argmax(1) == y.to(device)).type(torch.float).sum().item()
    
    loss /= (size/batch_size)
    hit /= size
    print(f"Test Error: \n Accuracy: {(100*hit):>0.1f}%, Avg loss: {loss:>8f}\n")

    return hit

In [ ]:
epochs = 20
train_loss1, test_acc1, test_acc2, test_acc3 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model1.train()
    train_loss1.append(train_loop(o_trainloader, model1, criterion, optimizer1))
    model1.eval()
    test_acc1.append(test_loop(o_testloader, model1, criterion))
    test_acc2.append(test_loop(s_testloader, model1, criterion))
    test_acc3.append(test_loop(s4_testloader, model1, criterion))

In [ ]:
epochs = 20
train_loss2, test_acc4, test_acc5, test_acc6 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model2.train()
    train_loss2.append(train_loop(s_trainloader, model2, criterion, optimizer2))
    model2.eval()
    test_acc4.append(test_loop(o_testloader, model2, criterion))
    test_acc5.append(test_loop(s_testloader, model2, criterion))
    test_acc6.append(test_loop(s4_testloader, model2, criterion))

In [ ]:
epochs = 20
train_loss3, test_acc7, test_acc8, test_acc9 = [], [], [], []
for i in range(epochs):
    print(f"Epoch {i + 1}-------------------------------")
    model3.train()
    train_loss3.append(train_loop(s4_trainloader, model3, criterion, optimizer3))
    model3.eval()
    test_acc7.append(test_loop(o_testloader, model3, criterion))
    test_acc8.append(test_loop(s_testloader, model3, criterion))
    test_acc9.append(test_loop(s4_testloader, model3, criterion))

In [ ]:
print("Result for Testing with original datasets")
print(max(test_acc1), max(test_acc2), max(test_acc3))

print("Result for Testing with original datasets")
print(max(test_acc4), max(test_acc5), max(test_acc6))

print("Result for Testing with original datasets")
print(max(test_acc7), max(test_acc8), max(test_acc9))
